In [ ]:
! pip install --no-index --find-links /kaggle/input/pycocotools/wheelhouse pycocotools

In [ ]:
import sys
sys.path.append("/kaggle/input/detection")

In [ ]:
import os
import glob
import numpy as np
import torch
from PIL import Image
from pathlib import Path
import cv2
from engine import train_one_epoch, evaluate
from data_process import img_showmask
import utils
import torch.utils.data as data
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import tifffile as tiff
import pandas as pd

In [ ]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, imgs, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = imgs
        self.name_indices = [os.path.splitext(os.path.basename(i))[0] for i in imgs]

    def __getitem__(self, idx):
        # load images and masks
        img_path = self.imgs[idx]
        name = self.name_indices[idx]
        array = tiff.imread(img_path)
        img = Image.fromarray(array)
        
        img, _ = self.transforms(img, img)

        return img, name

    def __len__(self):
        return len(self.imgs)

In [ ]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights = None , weights_backbone = None)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
all_imgs = glob.glob('/kaggle/input/hubmap-hacking-the-human-vasculature/test/*.tif')
dataset_test = PennFudanDataset(all_imgs, get_transform(train=False))


data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

In [ ]:
ids = []
heights = []
widths = []
prediction_strings = []

In [ ]:
model_best = get_model_instance_segmentation(num_classes=2).to(device)

model_best.load_state_dict(torch.load("/kaggle/input/output/epoch19.ckpt"))
model_best.eval()

for img, name in dataset_test:
    prediction = model_best([img.to(device)])
    pred_string = ""
    
    for idx in range(len(prediction[0]['masks'])):
        mask = prediction[0]['masks'][idx].detach().permute(1,2,0).cpu().numpy().astype(bool)
        score = prediction[0]['scores'][idx].detach().cpu().numpy()
        
        if score > 0.5:
            binary_mask = encode_binary_mask(mask).decode("utf-8")
            
            if idx>0:
                pred_string += " ";
                
            pred_string += f"0 {score} {binary_mask}"
    
    b, h, w = img.shape
    ids.append(name)
    heights.append(h)
    widths.append(w)
    prediction_strings.append(pred_string)

In [ ]:
submission = pd.DataFrame()
submission['id'] = ids
submission['height'] = heights
submission['width'] = widths
submission['prediction_string'] = prediction_strings
submission = submission.set_index('id')
submission.to_csv("submission.csv")
submission.head()